In [1]:
import socket
import json
import os

def load_eflint_file(path):
    with open(path, 'r') as in_file:
        code_lines = []
        for line in in_file.readlines():
            if len(line) and not line.startswith('#') and not line.startswith('//'):
                code_lines.append(line)
            if line.strip() == '####':
                break
        code = "\n".join(code_lines).strip().replace("\n\n", "\n")
    return code

def run_command(stream, command):
    message = (json.dumps(command) + '\n').encode('utf-8')
    stream.send(message)
    
def read_output(stream):
    result = ''
    partial_result = ''
    while True:
        partial_result = stream.recv(1024).decode('utf-8')
        result += partial_result
        if partial_result.strip().endswith('}') and result.count('{') == result.count('}'):
            parsed = json.loads(result)
            return parsed
        
def check_violations_or_errors(output):
    if output['violations']:
        raise ValueError(output['violations'])
    if output['errors']:
        raise ValueError(output['errors'])

class eFLINTServerConnection:
    
    def __init__(self, host, port):
        self.host = host
        self.port = port
        
    def run(self, command, pprint=False):
        stream = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        stream.connect((self.host, self.port))
        run_command(stream, command)
        result = read_output(stream)
        stream.close()
        if pprint:
            print(json.dumps(result, indent=4))
        else:
            return result
        
    def run_phrase(self, phrase, *args, **kwargs):
        return self.run(command={
            'command': 'phrase',
            'text': phrase
        }, *args, **kwargs)
    
code = load_eflint_file(os.path.join('..', 'data', 'in-flight.eflint'))
server = eFLINTServerConnection('eflint', 5000)
# Revert to an initial state
server.run({
    'command': 'revert',
    'value': 1
})
# Execute the domain definitions, acts and facts
server.run({
    'command': 'phrase',
    'text': code
}, pprint=True)
# Now create an UAV (called "Drone")
server.run({
    'command': 'create',
    'value': {
        'fact-type': 'uav',
        'value': 'Drone'
    }
})
# Create a remote pilot (called "RemotePilot")
server.run({
    'command': 'create',
    'value': {
        'fact-type': 'remotepilot',
        'value': 'RemotePilot'
    }
})
# Make remote pilot a controller of the drone
server.run({
    'command': 'phrase',
    'text': '+is-controller-of(remotepilot=RemotePilot, uav=Drone)'
})
# Make the start-flight method callable
server.run({
    'command': 'create',
    'value': {
        'fact-type': 'start-flight',
        'value': [
            {'fact-type': 'remotepilot', 'value': 'RemotePilot'},
            {'fact-type': 'uav', 'value': 'Drone'}
        ]
    }
})
# Make the end-flight method callable
server.run({
    'command': 'create',
    'value': {
        'fact-type': 'end-flight',
        'value': [
            {'fact-type': 'remotepilot', 'value': 'RemotePilot'},
            {'fact-type': 'uav', 'value': 'Drone'}
        ]
    }
})
print('Initialization complete!')

{
    "all-duties": [],
    "new-duties": [],
    "response": "success",
    "new-disabled-transitions": [
        {
            "fact-type": "end-flight",
            "arguments": [
                {
                    "fact-type": "remotepilot",
                    "value": "RemotePilot",
                    "textual": "remotepilot(\"RemotePilot\")"
                },
                {
                    "fact-type": "uav",
                    "value": "Drone",
                    "textual": "uav(\"Drone\")"
                }
            ],
            "textual": "end-flight(remotepilot(\"RemotePilot\"),uav(\"Drone\"))"
        },
        {
            "fact-type": "start-flight",
            "arguments": [
                {
                    "fact-type": "remotepilot",
                    "value": "RemotePilot",
                    "textual": "remotepilot(\"RemotePilot\")"
                },
                {
                    "fact-type": "uav",
                    "value": "

In [2]:
server.run({
    'command': 'phrase',
    'text': '+uav-within-earth-distance(uav=Drone, distance=130)'
})
#check_violations_or_errors(output)
output = server.run_phrase('start-flight(remotepilot=RemotePilot, uav=Drone)')
check_violations_or_errors(output)

ValueError: [{'violation': 'duty', 'value': {'fact-type': 'is-in-vlos', 'arguments': [{'fact-type': 'uav', 'value': 'Drone', 'textual': 'uav("Drone")'}, {'fact-type': 'claimant', 'value': 'RemotePilot', 'textual': 'claimant("RemotePilot")'}], 'textual': 'is-in-vlos(uav("Drone"),claimant("RemotePilot"))'}}]

In [3]:
server.run({
    'command': 'status'
}, pprint=True)

{
    "all-duties": [
        {
            "fact-type": "is-in-vlos",
            "arguments": [
                {
                    "fact-type": "uav",
                    "value": "Drone",
                    "textual": "uav(\"Drone\")"
                },
                {
                    "fact-type": "claimant",
                    "value": "RemotePilot",
                    "textual": "claimant(\"RemotePilot\")"
                }
            ],
            "textual": "is-in-vlos(uav(\"Drone\"),claimant(\"RemotePilot\"))"
        }
    ],
    "new-duties": [
        {
            "fact-type": "is-in-vlos",
            "arguments": [
                {
                    "fact-type": "uav",
                    "value": "Drone",
                    "textual": "uav(\"Drone\")"
                },
                {
                    "fact-type": "claimant",
                    "value": "RemotePilot",
                    "textual": "claimant(\"RemotePilot\")"
                }
  